# Get map locations

## Read dataframe from csv

In [10]:
import pandas as pd

df = pd.read_csv("export_files\gas_prices_20220415_153817.csv")
df.set_index("place_id",inplace=True)
df.dropna(subset=["latitude","longitude"],inplace=True)
df

,name,cree_id,latitude,longitude,regular,premium,diesel
place_id,,,,,,,
2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.92140,22.49,24.99,21.99
2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.30370,-99.74484,21.25,23.19,23.09
2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.45140,17.28,19.20,NaN
2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.30420,21.79,24.39,23.54
2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,17.79,19.99,NaN
...,...,...,...,...,...,...,...
27758,SUMINISTRO DE COMBUSTIBLES EL SEIS S. DE R.L. ...,PL/23961/EXP/ES/2022,20.88375,-102.70250,NaN,NaN,NaN
27759,GASOLINERA SACSALUM SA DE CV,PL/23960/EXP/ES/2022,16.32370,-91.97680,NaN,NaN,NaN
27764,"AEROPUERTO INTERNACIONAL FELIPE ANGELES, SOCIE...",PL/23974/EXP/ES/2022,19.72579,-99.03263,NaN,NaN,NaN


## Getting 100 nearest gas stations function 3

In [12]:
def distances(latitude: float, longitude: float):
    import geopy.distance
    distances = []
    for idx in df.index:
        distances.append(
            geopy.distance.geodesic(
                (latitude, longitude),
                (df.loc[idx,"latitude"], df.loc[idx,"longitude"])
                ).km
            )
    distances = pd.DataFrame(distances, columns=["distances"])
    distances["place_id"] = df.index
    distances.set_index("place_id",inplace=True)
    distances_idx = distances.sort_values(by="distances").index[1:100]
    return list(distances_idx)

## Plot map with locations

In [13]:
import folium

# Select location
center = [25.69651, -100.31676]

# gas type ["regular","premium","diesel"]
gas_type = "regular"

map = folium.Map(
    location = [center[0],center[1]],
    zoom_start = 13
    )

distance_index = distances(center[0],center[1])

max_price = df.loc[distance_index, gas_type].max()
min_price = df.loc[distance_index, gas_type].min()

thresold1 = df.loc[distance_index,gas_type].quantile(0.25)
thresold2 = df.loc[distance_index,gas_type].quantile(0.75)
#thresold1 = (max_price-min_price)*0.25 + min_price
#thresold2 = (max_price-min_price)*0.75 + min_price

for idx in distance_index:
    price = df.loc[idx,gas_type]
    
    if price < thresold1 and price > min_price:
        set_color = "lightgreen"
        
    elif price >= thresold1 and price <= thresold2:
        set_color = "blue"
        
    elif price > thresold2 and price < max_price:
        set_color = "lightred"
        
    if price == min_price:
        set_color = "green"
        
    elif price == max_price:
        set_color = "red"

    folium.Marker(
        location=[df.loc[idx,"latitude"], df.loc[idx,"longitude"]],
        popup=df.loc[idx,"name"]+"\nRegular: "+str(df.loc[idx,"regular"])+"\nPremium: "+ \
            str(df.loc[idx,"premium"])+"\nDiesel: "+str(df.loc[idx,"diesel"]),
        icon=folium.Icon(color=set_color)
    ).add_to(map)

map


### Export map

In [18]:
from datetime import datetime
import re

#Creating name of file based in date
date_ex = re.sub("[-]", "", str(datetime.today()))
date_ex = re.sub("[ ]", "_", date_ex)
date_ex = re.sub("[:]", "", date_ex)
date_ex = date_ex[:-7]

map.save("export_files\map_{}.html".format(date_ex))